In [1]:
import pandas as pd
import requests, json
import time
import os


In [2]:
df_marks_and_models = pd.read_csv('all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)
print('Кол-во марок: ', df_marks_and_models.shape[0])
display(df_marks_and_models.head(2))

Кол-во марок:  262


,Models
Mark,
AC,"['378_GT', 'ACE', 'ACECA', 'COBRA']"
ACURA,"['CL', 'CSX', 'EL', 'ILX', 'INTEGRA', 'LEGEND'..."


In [3]:
# кол-во объявлений на сайте на 9.9.2020 
dict_of_num_cars_of_models ={'AC':2, 'Acura':94, 'Alfa Romeo':65, 'Alpina':10, 
                             'AMC':2, 'Ariel':1, 'Asia':1, 'Aston Martin':25, 
                             'Audi':4093, 'Austin Healey':1, 'Bajaj':3, 
                             'Bentley':104, 'BMW':7423, 'Borgward':1, 
                             'Brilliance':10, 'Buick':17, 'BYD':22, 
                             'Cadillac':321, 'Changan':100, 'Chery':593, 
                             'Chevrolet':3670, 'Chrysler':316, 'Citroen':1142, 
                             'Dacia':9, 'Daewoo':1024, 'Daihatsu':47, 
                             'Daimler':5, 'Datsun':212, 'Delage':1, 'Derways':2, 
                             'DKW':1, 'Dodge':413, 'DongFeng':25, 'Doninvest':2, 
                             'DS':3, 'DW Hower':8, 'Eagle':1, 'Excalibur':1, 
                             'FAW':66, 'Ferrari':29, 'Fiat':206, 'Ford':4812, 
                             'Foton':2, 'GAC':14, 'Geely':302, 'Genesis':54, 
                             'GMC':26, 'Great Wall':420, 'Hafei':6, 'Haima':12, 
                             'Haval':110, 'Hawtai':4, 'Heinkel':1, 'Honda':1179, 
                             'HuangHai':1, 'Hummer':70, 'Hyundai':5712, 
                             'Infiniti':1041, 'Innocenti':1, 'Iran Khodro':21, 
                             'Isuzu':43, 'JAC':8, 'Jaguar':308, 'Jeep':466, 'JMC':1, 
                             'Kia':5595, 'LADA (ВАЗ)':8935, 'Lamborghini':14, 
                             'Lancia':8, 'Land Rover':1616, 'Lexus':985, 
                             'Lifan':295, 'Lincoln':62, 'Lotus':1, 'Luxgen':9, 
                             'Mahindra':1, 'Maserati':54, 'Maybach':7, 
                             'Mazda':2311, 'McLaren':5, 'Mercedes-Benz':8005, 
                             'Mercury':16, 'Metrocab':2, 'MG':10, 'MINI':313, 
                             'Mitsubishi':3935, 'Mitsuoka':3, 'Nissan':5776, 
                             'Oldsmobile':10, 'Opel':3131, 'Packard':2, 
                             'Peugeot':1636, 'Plymouth':13, 'Pontiac':38, 
                             'Porsche':714, 'Proton':3, 'PUCH':1, 'Ravon':112, 
                             'Renault':2771, 'Rolls-Royce':55, 'Rover':69, 
                             'Saab':147, 'Saturn':9, 'Scion':9, 'SEAT':77, 
                             'Shanghai Maple':1, 'Skoda':2810, 'Smart':89, 
                             'SsangYong':718, 'Subaru':934, 'Suzuki':878, 
                             'Tatra':5, 'Tesla':43, 'Tianma':2, 'Toyota':4648, 
                             'Triumph':1, 'Vauxhall':1, 'Volkswagen':7109, 
                             'Volvo':2069, 'Vortex':91, 'Wartburg':1, 'Willys':2, 
                             'Zotye':15, 'ZX':4, 'ГАЗ':935, 'Гоночный болид':2, 
                             'ЗАЗ':231, 'ЗИЛ':2, 'ИЖ':129, 'ЛуАЗ':45, 
                             'Москвич':198, 'СМЗ':4, 'ТагАЗ':68, 'УАЗ':1107}

In [4]:
# уберем модели с кол-вом объявлений ниже 300 и повысим регистр индексов 
list_keys_to_delete = []
for mark in dict_of_num_cars_of_models.keys():
    if dict_of_num_cars_of_models[mark] < 300:
        list_keys_to_delete.append(mark)
    
for key in list_keys_to_delete: del dict_of_num_cars_of_models[key]

for mark in list(dict_of_num_cars_of_models.keys()):
    mark_up = mark.upper()
    dict_of_num_cars_of_models[mark_up]=dict_of_num_cars_of_models.pop(mark)

In [5]:
# посмотрим на список получившихся моделей
print(*dict_of_num_cars_of_models, sep=',')

AUDI,BMW,CADILLAC,CHERY,CHEVROLET,CHRYSLER,CITROEN,DAEWOO,DODGE,FORD,GEELY,GREAT WALL,HONDA,HYUNDAI,INFINITI,JAGUAR,JEEP,KIA,LADA (ВАЗ),LAND ROVER,LEXUS,MAZDA,MERCEDES-BENZ,MINI,MITSUBISHI,NISSAN,OPEL,PEUGEOT,PORSCHE,RENAULT,SKODA,SSANGYONG,SUBARU,SUZUKI,TOYOTA,VOLKSWAGEN,VOLVO,ГАЗ,УАЗ


In [6]:
# Удалим отечественные модели
for key in ['LADA (ВАЗ)','ГАЗ','УАЗ']: del dict_of_num_cars_of_models[key]

In [7]:
# проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300:= {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300:= 36
список марок, которые нужно проверить := GREAT WALL LAND ROVER MERCEDES-BENZ SSANGYONG


In [8]:
dict_of_num_cars_of_models['GREAT WALL']

420

In [9]:
# исправим несоответствия
dict_of_num_cars_of_models['GREAT_WALL']=dict_of_num_cars_of_models.pop('GREAT WALL')
dict_of_num_cars_of_models['LAND_ROVER']=dict_of_num_cars_of_models.pop('LAND ROVER')
dict_of_num_cars_of_models['MERCEDES']=dict_of_num_cars_of_models.pop('MERCEDES-BENZ')
dict_of_num_cars_of_models['SSANG_YONG']=dict_of_num_cars_of_models.pop('SSANGYONG')

In [10]:
# еще раз проверим что названия моделей совпадаю с названиями каталога сайта auto.Ru
list_marks_df = list(df_marks_and_models.index)
list_marks_to_check = []
for mark in dict_of_num_cars_of_models.keys():
    if mark not in list_marks_df:
        list_marks_to_check.append(mark)
print(f'всего марок с кол-вом объявлений более 300 := {len(dict_of_num_cars_of_models)}')
print('список марок, которые нужно проверить :=', *list_marks_to_check)

всего марок с кол-вом объявлений более 300 := 36
список марок, которые нужно проверить :=


In [11]:
# запишем список релевантных марок для парсинга
list_of_relevant_marks = list(dict_of_num_cars_of_models.keys())

In [12]:
sum_of_ads = sum(dict_of_num_cars_of_models.values())
print(f"примерное кол-во строк после парсинга должно быть := {sum_of_ads}")

примерное кол-во строк после парсинга должно быть := 89189


In [13]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Connection': 'keep-alive',
    'Content-Length': '137',
    'content-type': 'application/json',
    'Cookie': '_csrf_token=1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24; autoru_sid=a%3Ag5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270%7C1580931467355.604800.8HnYnADZ6dSuzP1gctE0Fw.cd59AHgDSjoJxSYHCHfDUoj-f2orbR5pKj6U0ddu1G4; autoruuid=g5e3b198b299o5jhpv6nlk0ro4daqbpf.fa3630dbc880ea80147c661111fb3270; suid=48a075680eac323f3f9ad5304157467a.bc50c5bde34519f174ccdba0bd791787; from_lifetime=1580933172327; from=yandex; X-Vertis-DC=myt; crookie=bp+bI7U7P7sm6q0mpUwAgWZrbzx3jePMKp8OPHqMwu9FdPseXCTs3bUqyAjp1fRRTDJ9Z5RZEdQLKToDLIpc7dWxb90=; cmtchd=MTU4MDkzMTQ3MjU0NQ==; yandexuid=1758388111580931457; bltsr=1; ',
    'Host': 'auto.ru',
    'origin': 'https://auto.ru/cars/all/',
    'Referer': 'https://auto.ru/cars/all/',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
    'x-client-app-version': '202002.03.092255',
    'x-client-date': '1580933207763',
    'x-csrf-token': '1c0ed592ec162073ac34d79ce511f0e50d195f763abd8c24',
    'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
    'x-requested-with': 'fetch'
}

In [14]:
url = "https://auto.ru/-/ajax/desktop/listing/"

In [2]:
# посмотрим какие столбцы нам нужны 
test = pd.read_csv('1test.csv')
print('Кол-во столбцов теста:', test.shape[1])
print('Названия столбцов теста:', list(test.columns))

# вывод всех столбцов датафрейма
pd.set_option('display.max_columns', None)
test.head(2)

Кол-во столбцов теста: 32
Названия столбцов теста: ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement', 'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [17]:
# сделаем тестовый вызов запроса для BMW модели M6 и выведем его на экран, чтобы по нему можно было сделать функцию для создания 23 необходимых столбцов
model = 'M6'
param = {
                'catalog_filter' : [{"mark": 'BMW', "model": model}],
                'section': "used",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"600",
                "geo_radius":200,
                "geo_id":[213],
                'page': 1
            }

response = requests.post(url, json=param, headers = HEADERS)
data = response.json()
for elem in data['offers']:
    print(elem)
    break

{'availability': 'IN_STOCK', 'category': 'cars', 'color_hex': '040001', 'delivery_info': {}, 'description': 'Автомобиль с завода competition. \nПолная комплектация за исключением подогрева руля. \nВыполнен полный детейлинг  ,защита сидений ,плёнка передних фар,бампера и капота. \nНовая резина Michelin. \nУстановлен «Цезарь-трекер»\nСостояние автомобиля идеальное ,не битая и не крашенная. \nВыполнены все ТО, установлен Stage1 и педаль на улице Осенняя.\nВ зимний период не эксплуатировалась\nлюбые проверки у официальных дилеров в г.Москве. \n\n\n\nПерекупщикам и автосалонам просьба не беспокоить. \n', 'discount_options': {}, 'documents': {'owners_number': 3, 'pts_original': True, 'custom_cleared': True, 'purchase_date': {'year': 2020, 'month': 2}, 'year': 2015, 'vin': 'WBS**************', 'pts': 'ORIGINAL', 'license_plate': '******|77', 'vin_resolution': 'OK'}, 'is_favorite': False, 'old_category_id': 15, 'owner_expenses': {'transport_tax': {'tax_by_year': 86250, 'year': 2021, 'rid': 213

In [18]:
# строка для проверки содержания элемента запроса
elem['additional_info']['hidden']

False

In [19]:
def utils_clean_values_dict(d_dict):
    for elem in d_dict:
        d_dict[elem] = 0
    return

In [20]:
dict_columns = {'bodyType':0, 'brand':0, 'color':0, 'fuelType':0, 'modelDate':0, 
                'name':0, 'numberOfDoors':0, 'productionDate':0, 
                'vehicleConfiguration':0, 'vehicleTransmission':0, 
                'engineDisplacement':0, 'enginePower':0, 'description':0, 
                'mileage':0, 'Комплектация':0, 'Привод':0, 'Руль':0, 
                'Состояние':0, 'Владельцы':0, 'ПТС':0, 'Таможня':0, 
                'Владение':0, 'price':0}

model = 'M6'
# создаем функцию для обработки 23 необходимых столбцов
def from_elem_to_dict(d_elem, d_dict):
    utils_clean_values_dict(d_dict)
    
    try: d_dict['bodyType'] = d_elem['vehicle_info']['configuration']['human_name']
    except: d_dict['bodyType'] = None

    try: d_dict['brand'] = d_elem['vehicle_info']['mark_info']['code']
    except: d_dict['brand'] = None

    try: d_dict['color'] = d_elem['color_hex']
    except: d_dict['color'] = None

    try: d_dict['fuelType'] = d_elem['lk_summary'].split()[-1]
    except: d_dict['fuelType'] = None

    try: d_dict['modelDate'] = d_elem['vehicle_info']['super_gen']['year_from']
    except: d_dict['modelDate'] = None

    try: d_dict['name'] =  d_elem['vehicle_info']['tech_param']['human_name']
    except: d_dict['name'] = None

    try: d_dict['numberOfDoors'] = d_elem['vehicle_info']['configuration']['doors_count']
    except: d_dict['numberOfDoors'] = None

    try: d_dict['productionDate'] = d_elem['documents']['year']
    except: d_dict['productionDate'] = None

    try: d_dict['vehicleTransmission'] = d_elem['vehicle_info']['tech_param']['transmission']
    except: d_dict['vehicleTransmission'] = None

    try: d_dict['engineDisplacement'] = d_elem['vehicle_info']['tech_param']['human_name'].split()[0]
    except: d_dict['engineDisplacement'] = None

    try: d_dict['vehicleConfiguration'] = elem['vehicle_info']['configuration']['body_type']+' '+d_dict['vehicleTransmission']+' '+d_dict['engineDisplacement']
    except: d_dict['vehicleConfiguration'] = None

    try: d_dict['enginePower'] = d_elem['vehicle_info']['tech_param']['power']
    except: d_dict['enginePower'] = None

    try: d_dict['description'] = d_elem['description']
    except: d_dict['description'] = None

    try: d_dict['mileage'] = d_elem['state']['mileage']
    except: d_dict['mileage'] = None

    try: d_dict['Комплектация'] = d_elem['vehicle_info']['complectation']
    except: d_dict['Комплектация'] = None

    try: d_dict['Привод'] = d_elem['lk_summary'].split(', ')[-2]
    except: d_dict['Привод'] = None

    try: d_dict['Руль'] = d_elem['vehicle_info']['steering_wheel']
    except: d_dict['Руль'] = None

    # 'Состояние' не найдено
    d_dict['Состояние'] = None

    try: d_dict['Владельцы'] = d_elem['documents']['owners_number']
    except: d_dict['Владельцы'] = None

    try: d_dict['ПТС'] = d_elem['documents']['pts']
    except: d_dict['ПТС'] = None

    try: d_dict['Таможня'] = d_elem['documents']['custom_cleared']
    except: d_dict['Таможня'] = None

    try: d_dict['Владение'] =  d_elem['documents']['purchase_date']
    except: d_dict['Владение'] = None


    try: d_dict['price'] = d_elem['price_info']['RUR']
    except: d_dict['price'] = None

    # дата размещения объявления решил добавить 
    try: d_dict['start_date'] =  d_elem['additional_info']['hot_info']['start_time']
    except: d_dict['start_date'] = None

    # статус объявления 
    try: d_dict['hidden'] =  d_elemelem['additional_info']['hidden']
    except: d_dict['hidden'] = None

    # модель 
    d_dict['model'] =  model

    return

from_elem_to_dict(elem, dict_columns)

In [21]:
display(pd.DataFrame(dict_columns).head(2))
test[test['enginePower']=='560 N12'].head(2)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
id,Кабриолет,BMW,040001,бензин,2012,Competition Pack 4.4 AMT (575 л.с.),2,2015,CABRIO ROBOT Competition,ROBOT,Competition,575,Автомобиль с завода competition. \nПолная комп...,39800,0,задний,LEFT,None,3,ORIGINAL,True,NaN,5100000,2021-01-29T15:13:06Z,None,M6
year,Кабриолет,BMW,040001,бензин,2012,Competition Pack 4.4 AMT (575 л.с.),2,2015,CABRIO ROBOT Competition,ROBOT,Competition,575,Автомобиль с завода competition. \nПолная комп...,39800,NaN,задний,LEFT,None,3,ORIGINAL,True,2020.0,5100000,2021-01-29T15:13:06Z,None,M6


,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
2385,лифтбек,AUDI,https://auto.ru/cars/used/sale/audi/rs7/110038...,серый,NaN,Audi Rs7 VSperformance Stage3\nНовый усиленный...,4.0 LTR,560 N12,NaN,бензин,https://autoru.naydex.net/ys1kR7800/fbd964zfcB...,39900,2013,"{""code"":""RS7"",""name"":""RS 7"",""ru_name"":""РС7"",""m...",RS7,4.0 AT (560 л.с.) 4WD,5,1603121542,RUB,2013,1100381316,"{""id"":""20045126"",""displacement"":3993,""engine_t...",LIFTBACK AUTOMATIC 4.0,автоматическая,EUROPEAN,3 или более,2 года и 4 месяца,Оригинал,полный,Левый,Не требует ремонта,Растаможен
2576,лифтбек,AUDI,https://auto.ru/cars/used/sale/audi/rs7/110125...,чёрный,NaN,Внимание! Только для клиентов AVILON Автомобил...,4.0 LTR,560 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://autoru.naydex.net/ys1kR7800/fbd964zfcB...,181055,2014,"{""code"":""RS7"",""name"":""RS 7"",""ru_name"":""РС7"",""m...",RS7,4.0 AT (560 л.с.) 4WD,5,1603122317,RUB,2015,1101251163,"{""id"":""20183797"",""displacement"":3993,""engine_t...",LIFTBACK AUTOMATIC 4.0,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [22]:
auto_ru = {}

iter_of_ads, iter_of_marks, old_iter_of_ads, sum_ads, sum_ads_from_dict = 0,0,0,0,0
space = ' '
start_time_global = time.clock()

for mark in list_of_relevant_marks:
    start_time = time.clock()
    list_of_models = json.loads(df_marks_and_models.loc[mark, 'Models'].replace("'", '"'))
    for model in list_of_models:
        for num_page in range(1, 200):
            param = {
                'catalog_filter' : [{"mark": mark, "model": model}],
                'section': "all",
                'category': "cars",
                'sort': "fresh_relevance_1-desc",
                "top_days":"900",
                "geo_radius":200,
                "geo_id":[213],
                'page': num_page
            }
            response = requests.post(url, json=param, headers = HEADERS)
            data = response.json()

            if data['offers'] != []:
                for elem in data['offers']:
                    from_elem_to_dict(elem, dict_columns)
                    auto_ru[iter_of_ads] = dict_columns.copy()
                    iter_of_ads += 1
            else:
                break
    iter_of_marks +=1
    temp_1 = iter_of_ads-old_iter_of_ads
    temp_2 = dict_of_num_cars_of_models[mark]
    temp_3 = temp_1/temp_2
    temp_4 = space*abs(13-len(mark)-len(str(iter_of_marks)))
    temp_5 = int(time.clock() - start_time)
    temp_8 = round(temp_3*100,2)
    temp_7 = space*abs(18-len(str(temp_1))-len(str(temp_2))-len(str(temp_8))-len(str(temp_5)))
    temp_9 = int(time.clock() - start_time_global)
    

    sum_ads += temp_1
    sum_ads_from_dict += temp_2
    temp_def = sum_ads/sum_ads_from_dict
    temp_6 = (sum_of_ads*temp_def - iter_of_ads)*temp_9/sum_ads


    print (f'{iter_of_marks}.{mark} {temp_4} ушло:{temp_5} сек., стр. {temp_1} из {temp_2} {temp_8}%,{temp_7}всего стр. {iter_of_ads}, прогноз {int(temp_6/60)} мин.')

    old_iter_of_ads=iter_of_ads

print (f'Всего спарсили := {iter_of_ads} строк, на это ушло: {int((time.clock() - start_time_global)/60)} мин., ')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


1.AUDI          ушло:58 сек., стр. 2555 из 4093 62.42%,   всего стр. 2555, прогноз 20 мин.
2.BMW           ушло:136 сек., стр. 5982 из 7423 80.59%,  всего стр. 8537, прогноз 21 мин.
3.CADILLAC      ушло:18 сек., стр. 317 из 321 98.75%,     всего стр. 8854, прогноз 23 мин.
4.CHERY         ушло:16 сек., стр. 355 из 593 59.87%,     всего стр. 9209, прогноз 23 мин.
5.CHEVROLET     ушло:68 сек., стр. 2153 из 3670 58.66%,   всего стр. 11362, прогноз 22 мин.
6.CHRYSLER      ушло:15 сек., стр. 209 из 316 66.14%,     всего стр. 11571, прогноз 23 мин.
7.CITROEN       ушло:31 сек., стр. 843 из 1142 73.82%,    всего стр. 12414, прогноз 23 мин.
8.DAEWOO        ушло:18 сек., стр. 554 из 1024 54.1%,     всего стр. 12968, прогноз 23 мин.
9.DODGE         ушло:18 сек., стр. 264 из 413 63.92%,     всего стр. 13232, прогноз 23 мин.
10.FORD         ушло:72 сек., стр. 2884 из 4812 59.93%,   всего стр. 16116, прогноз 20 мин.
11.GEELY        ушло:8 сек., стр. 223 из 302 73.84%,      всего стр. 16339, прогноз 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [24]:
# запись датасета
df_auto_ru = pd.DataFrame(auto_ru).T
df_auto_ru.to_csv('all_auto_ru_01_02_2021.csv', index=False)
print(f'кол-во строк датасета:= {len(df_auto_ru)}')

кол-во строк датасета:= 65351


In [26]:
# визуальный контроль
df_auto_ru.sample(10)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
20616,Седан,HYUNDAI,040001,бензин,2010,1.6 AT (123 л.с.),4,2011,SEDAN AUTOMATIC 1.6,AUTOMATIC,1.6,123,Машина в отличном состоянии! Все жидкости поме...,82200,{'id': '0'},передний,LEFT,None,2,ORIGINAL,True,None,490000,2021-01-26T10:27:23Z,None,SOLARIS
23975,Седан,KIA,FAFBFB,бензин,2018,2.4 AT (188 л.с.),4,2018,SEDAN AUTOMATIC 2.4,AUTOMATIC,2.4,188,"Автомобиль новый, недостатков нет. Салон чисты...",32154,"{'id': '21342410', 'name': 'GT Line', 'availab...",передний,LEFT,None,1,ORIGINAL,True,"{'year': 2018, 'month': 2}",1800000,2020-12-10T18:11:19Z,None,OPTIMA
11013,Внедорожник 5 дв.,CHEVROLET,007F00,бензин,2009,1.7 MT (80 л.с.) 4WD,5,2010,ALLROAD_5_DOORS MECHANICAL 1.7,MECHANICAL,1.7,80,"Машина в хорошем состояние, езжу каждый день. ...",139600,{'id': '0'},полный,LEFT,None,1,ORIGINAL,True,"{'year': 2010, 'month': 3}",330000,2020-11-29T13:22:09Z,None,NIVA
30052,Внедорожник 5 дв.,MITSUBISHI,CACECB,бензин,2012,1.6 MT (117 л.с.),5,2014,ALLROAD_5_DOORS MECHANICAL 1.6,MECHANICAL,1.6,117,Только до 31 января - максимально выгодные пре...,98222,{'id': '0'},передний,LEFT,None,1,ORIGINAL,True,None,789000,2021-01-19T04:35:36Z,None,ASX
53475,Внедорожник 5 дв.,VOLKSWAGEN,0000CC,бензин,2011,1.4 MT (150 л.с.) 4WD,5,2011,ALLROAD_5_DOORS MECHANICAL 1.4,MECHANICAL,1.4,150,"Ласточка покупалась в январе 2018,с пробегом 5...",82000,{'id': '0'},полный,LEFT,None,2,ORIGINAL,True,None,830000,2020-12-17T13:13:43Z,None,TIGUAN
31153,Седан,MITSUBISHI,CACECB,бензин,2005,1.6 MT (98 л.с.),4,2007,SEDAN MECHANICAL 1.6,MECHANICAL,1.6,98,Все вопросы по телефону. Перекупам и автосалон...,297000,{'id': '0'},передний,LEFT,None,3,ORIGINAL,True,"{'year': 2014, 'month': 6}",260000,2020-09-22T07:14:54Z,None,LANCER
51708,Универсал 5 дв.,VOLKSWAGEN,CACECB,газ,2005,EcoFuel 1.4 AMT (150 л.с.),5,2009,WAGON_5_DOORS ROBOT EcoFuel,ROBOT,EcoFuel,150,Машина полостью на ходу. В прошлом году произв...,223000,{'id': '0'},передний,LEFT,None,3,ORIGINAL,True,"{'year': 2017, 'month': 1}",390000,2019-06-14T17:41:37Z,None,PASSAT
5495,Седан Long,BMW,0000CC,бензин,1998,740Li 4.4 AT (286 л.с.),4,1999,SEDAN AUTOMATIC 740Li,AUTOMATIC,740Li,286,Продам BMW 740il e38.\nАвтомобиль в хорошем со...,221000,{'id': '0'},задний,LEFT,None,3,ORIGINAL,True,None,450000,2020-09-10T19:00:59Z,None,7ER
62463,Внедорожник 5 дв.,MERCEDES,97948F,дизель,2019,400 d 2.9d AT (330 л.с.) 4WD,5,2020,ALLROAD_5_DOORS AUTOMATIC 400,AUTOMATIC,400,330,Только в декабре!\nНовогодние подарки от крупн...,0,"{'id': '21829849', 'name': 'GLE 400 d 4MATIC',...",полный,LEFT,None,None,ORIGINAL,True,None,9652056,2020-12-23T20:09:05Z,None,GLE_KLASSE_COUPE
50353,Хэтчбек 5 дв.,VOLKSWAGEN,CACECB,бензин,2008,1.2 AMT (105 л.с.),5,2012,HATCHBACK_5_DOORS ROBOT 1.2,ROBOT,1.2,105,по кузу не достатки все по фото\nСвоевременно ...,66000,{'id': '0'},передний,LEFT,None,3,ORIGINAL,True,None,495000,2021-01-20T09:51:34Z,None,GOLF


Первоначальный парсинг показал, что на сайте много старых объявлений . Так как изначально за последние 90 дней мы получили 43% от всех объявлений по BMW, поэтому пришлось скачать за последние 900 дней, чтобы получить полные цифры заявленные на сайте. 

In [3]:
test.head()

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,74000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.2 AMT (105 л.с.),5,1603226273,RUB,2014,1100575026,"{""id"":""10373605"",""displacement"":1197,""engine_t...",LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,60563,2017,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 MT (110 л.с.),5,1603226277,RUB,2017,1100549428,"{""id"":""20913311"",""displacement"":1598,""engine_t...",LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,https://avatars.mds.yandex.net/get-autoru-vos/...,88000,2013,"{""code"":""SUPERB"",""name"":""Superb"",""ru_name"":""Су...",SUPERB,DSG 1.8 AMT (152 л.с.),5,1603226280,RUB,2014,1100658222,"{""id"":""20026323"",""nameplate"":""DSG"",""displaceme...",LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,95000,2013,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.6 AT (110 л.с.),5,1603226284,RUB,2014,1100937408,"{""id"":""20105521"",""displacement"":1598,""engine_t...",LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,https://autoru.naydex.net/o9DBXQ270/5ac010hAY0...,58536,2008,"{""code"":""OCTAVIA"",""name"":""Octavia"",""ru_name"":""...",OCTAVIA,1.8 AT (152 л.с.),5,1603226288,RUB,2012,1101037972,"{""id"":""4561004"",""displacement"":1798,""engine_ty...",LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


In [4]:
df_auto_ru = pd.read_csv('all_auto_ru_01_02_2021.csv')
print('Кол-во столбцов теста:', test.shape[1])
print('Названия столбцов теста:', list(test.columns))

# вывод всех столбцов датафрейма
pd.set_option('display.max_columns', None)
df_auto_ru.head(2)

Кол-во столбцов теста: 32
Названия столбцов теста: ['bodyType', 'brand', 'car_url', 'color', 'complectation_dict', 'description', 'engineDisplacement', 'enginePower', 'equipment_dict', 'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name', 'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency', 'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration', 'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня']


,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,start_date,hidden,model
0,Седан,AUDI,0000CC,бензин,1990.0,2.3 MT (133 л.с.),4.0,1993,SEDAN MECHANICAL 2.3,MECHANICAL,2.3,133.0,ЛОТ: 01237660\nFAVORIT MOTORS KIA Север\n\nВы ...,786115,{'id': '0'},передний,LEFT,NaN,3.0,ORIGINAL,True,NaN,93000.0,2021-01-07T10:58:58Z,NaN,100
1,Седан,AUDI,4A2197,бензин,1990.0,2.3 MT (133 л.с.),4.0,1993,SEDAN MECHANICAL 2.3,MECHANICAL,2.3,133.0,Машина в отличном состоянии . Двигатель и коро...,360000,{'id': '0'},передний,LEFT,NaN,4.0,DUPLICATE,True,"{'year': 2018, 'month': 6}",155000.0,2021-01-30T03:34:16Z,NaN,100


In [5]:
df_auto_ru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65351 entries, 0 to 65350
Data columns (total 26 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              65349 non-null  object 
 1   brand                 65351 non-null  object 
 2   color                 65351 non-null  object 
 3   fuelType              65351 non-null  object 
 4   modelDate             65349 non-null  float64
 5   name                  65349 non-null  object 
 6   numberOfDoors         65349 non-null  float64
 7   productionDate        65351 non-null  int64  
 8   vehicleConfiguration  65349 non-null  object 
 9   vehicleTransmission   65349 non-null  object 
 10  engineDisplacement    65349 non-null  object 
 11  enginePower           65349 non-null  float64
 12  description           62631 non-null  object 
 13  mileage               65351 non-null  int64  
 14  Комплектация          65351 non-null  object 
 15  Привод             

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

estimators = [
    ('cb',create_best_cat_boost_regressor()),
    ('rf',RandomForestRegressor(random_state=RANDOM_SEED))
]

reg = StackingRegressor(estimators=estimators)
reg.fit(X_train, np.log(y_train))

predict_test = np.exp(reg.predict(X_test))
predict_submission = np.exp(reg.predict(test))

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(3)